# 资讯分析师

## 准备工作

pip安装 `akshare` 和 `langchain` 库：

In [ ]:
%pip install --upgrade --quiet pandas akshare langchain-core langchain-openai langchain-community langchain-chroma

获取股票主营介绍数据：

In [ ]:
import pandas as pd
import akshare as ak

# 沪深京 A 股实时行情数据
stock_zh_a_spot_em_df = ak.stock_zh_a_spot_em()
# 股票对应的主营介绍数据
stock_zyjs_ths_dfs = []
for stock_code, stock_name in zip(stock_zh_a_spot_em_df['代码'], stock_zh_a_spot_em_df['名称']):
    try:
        stock_zyjs_ths_df = ak.stock_zyjs_ths(stock_code)
        stock_zyjs_ths_df['股票名称'] = stock_name
        stock_zyjs_ths_dfs.append(stock_zyjs_ths_df)
    except Exception as e:
        print(f"Error processing stock {stock_code}: {e}")
        continue
stock_zyjs_ths_df = pd.concat(stock_zyjs_ths_dfs)
stock_zyjs_ths_df.to_csv('data/stock_zyjs_ths.csv', index=False, quoting=1)

加载股票主营介绍数据：

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    file_path='data/stock_zyjs_ths.csv',
    csv_args={
        'delimiter': ',',
        'quotechar': '"',
    }
)

documents = loader.load()
print(f"Loaded {len(documents)} documents")

embedding股票主营介绍数据并存储到向量数据库：

In [ ]:
import os
import pandas as pd

from langchain_openai import AzureOpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
)

vector_store = Chroma(
    collection_name="stock_zyjs_ths",
    embedding_function=embeddings,
    persist_directory="data/verctor_store.db",
)

ids = pd.read_csv('data/stock_zyjs_ths.csv')['股票代码'].astype(str).tolist()

for i in range(0, len(ids), 100):
    print(f"Adding documents {i} to {i+100}")
    partial_ids = ids[i:i+100]
    partial_documents = documents[i:i+100]
    vector_store.add_documents(documents=partial_documents, ids=partial_ids)

## 资讯分析

连接向量数据库：

In [4]:
import os

from langchain_openai import AzureOpenAIEmbeddings
from langchain_chroma import Chroma

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_EMBEDDING_API_VERSION"],
)

vector_store = Chroma(
    collection_name="stock_zyjs_ths",
    embedding_function=embeddings,
    persist_directory="data/verctor_store.db",
)

定义检索器：

In [5]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5}
)

In [6]:
retriever.batch(
    [
        "上海机场的主营业务是什么？",
        "工商银行的主营业务是什么？",
    ],
)

[[Document(metadata={'row': 1134, 'source': 'data/stock_zyjs_ths.csv'}, page_content='股票代码: 600009\n股票名称: 上海机场\n主营业务: 从事航空运输地面服务及其他相关业务。\n产品类型: 航空及相关服务\n产品名称: 航空及相关服务\n经营范围: 国内外航空运输企业及旅客提供地面保障服务，经营出租机场内航空营业场所、商业场所和办公场所，国内贸易（除专项规定）；广告经营，经营其它与航空运输有关的业务；货运代理；代理报验；代理报关；长途客运站（限分支机构经营）；综合开发，经营国家政策许可的其它投资项目，停车场管理及停车延伸服务。（依法须经批准的项目，经相关部门批准后方可开展经营活动）。')],
 [Document(metadata={'row': 1462, 'source': 'data/stock_zyjs_ths.csv'}, page_content='股票代码: 601398\n股票名称: 工商银行\n主营业务: 从事公司和个人金融业务、资金业务、投资银行业务，并提供资产管理、信托、金融租赁及其他金融服务。\n产品类型: 银行业\n产品名称: 公司金融业务、个人金融业务、资金业务\n经营范围: 办理人民币存款、贷款；同业拆借业务；国内外结算；办理票据承兑、贴现、转贴现；各类汇兑业务；代理资金清算；提供信用证服务及担保；代理销售业务；代理发行、代理承销、代理兑付政府债券；代收代付业务；代理证券资金清算业务（银证转账）；保险兼业代理业务（有效期至2018年08月27日）；代理政策性银行、外国政府和国际金融机构贷款业务；保管箱服务；发行金融债券；买卖政府债券、金融债券；证券投资基金、企业年金托管业务；企业年金受托管理服务、年金账户管理服务；开放式基金的注册登记、认购、申购和赎回业务；资信调查、咨询、见证业务；贷款承诺；企业、个人财务顾问服务；组织或参加银团贷款；外汇存款；外汇贷款；外币兑换；出口托收及进口代收；外汇票据承兑和贴现；外汇借款；外汇担保；发行、代理发行、买卖或代理买卖股票以外的外币有价证券；自营、代客外汇买卖；外汇金融衍生业务；银行卡业务；电话银行、网上银行、手机银行业务；办理结汇、售汇业务；经国务院银行业监督管理机构批准的其他业务。')]]